In [29]:
import numpy as np

![image](Driveington_chart.png)

In [30]:
population = 5000

In [31]:
travel_time = lambda q_x, t0_x, Q_x: t0_x * (1 + pow(q_x/Q_x, 2))

In [32]:
class Section:
    def __init__(self, name):
        self.name = name
        self.neighbors = list()
    
    def make_connection(self, section):
        self.neighbors.append(section)

In [33]:
class Road:
    def __init__(self, origin, destination, free_flow, capacity):
        self.origin = origin
        self.destination = destination
        self.free_flow = free_flow
        self.capacity = capacity
        self.number_of_vehicles = 0
        self.randomness = np.random.rand()/5.0
        self.congestion = self.travel_time(self.number_of_vehicles, self.free_flow, self.capacity, self.randomness)
    
    def reset(self):
        self.number_of_vehicles = 0
        self.congestion = self.travel_time(self.number_of_vehicles, self.free_flow, self.capacity, self.randomness)
        
    def travel_time(self, q_x, t0_x, Q_x, randomness):
        incident = randomness > np.random.rand()
        congestion = t0_x * (1 + pow(q_x/Q_x, 2))
        if incident: return congestion * 5 # Road had an incident, congestion increased
        else: return congestion
    
    def add_car(self, influence):
        self.number_of_vehicles += influence
        self.congestion = self.travel_time(self.number_of_vehicles, self.free_flow, self.capacity, self.randomness)
        
    def remove_car(self, influence):
        self.number_of_vehicles -= influence
        self.congestion = self.travel_time(self.number_of_vehicles, self.free_flow, self.capacity, self.randomness)

In [34]:
class Vehicle:
    def __init__(self, vehicle_type, origin, destination):
        self.vehicle_type = vehicle_type
        self.origin = origin
        self.original_dest = destination
        self.destination = self.original_dest
        self.prev_location = None
        self.location = origin
        self.arrived = False
        
        self.randomness = np.random.rand()/10.0
        
    def reset(self):
        self.prev_location = None
        self.location = self.origin
        self.arrived = False
        self.destination = self.original_dest
        
    def move_to_section(self, section):
        if (section in self.location.neighbors):
            self.prev_location = self.location
            self.location = section
            if self.location == self.destination:
                self.arrived = True 
        else:
            raise ValueError("No road between %s and %s" % (location.name, section.name))
        
        if self.randomness > np.random.rand(): # Vehicle changed destination for some reason
            self.destination = np.random.choice((self.origin, np.random.choice(self.destination.neighbors)))

In [35]:
class City:
    def __init__(self, sections, randomness):
        self.sections = sections
        self.roads = list()
        self.randomness = randomness
        
    def reset(self):
        for r in self.roads: r.reset()
        for v in self.vehicles: v.reset()
       
    def build_road(self, section1_name, section2_name, free_flow, capacity):
        if section1_name != section2_name:
            section1, section2 = None, None
            for s in self.sections:
                if s.name == section1_name: section1 = s
                elif s.name == section2_name: section2 = s
            if not all((section1, section2)): return False
            
            section1.make_connection(section2)
            section2.make_connection(section1)
            self.roads.append(Road(section1, section2, free_flow, capacity))
            self.roads.append(Road(section2, section1, free_flow, capacity))
            return True
        else:
            print(section1_name, section2_name, "are the same.")
            return False
    
    def move_vehicle(self, vehicle, to_section):
        if np.random.rand() > self.randomness:
            if (not vehicle.arrived) and (to_section in vehicle.location.neighbors):

                vehicle_influence = 1 if vehicle.vehicle_type == "car" else 2
                prev_location, location = vehicle.prev_location, vehicle.location
                old_road, new_road = None, None
                for v in self.roads:
                    if (v.origin == prev_location) and (v.destination == location): old_road = v
                    elif (v.origin == location) and (v.destination == to_section): new_road = v
                vehicle.move_to_section(to_section)
                if old_road: old_road.remove_car(vehicle_influence)
                if not vehicle.arrived: new_road.add_car(vehicle_influence)
                return True

            elif vehicle.arrived:
                return True

            else:
                #print(to_section in vehicle.location.neighbors)
                return False
            
        else: # Vehicle did not move for some reason
            return True
        
    def get_congestion(self):
        congestions = []
        for road in self.roads: congestions.append(road.congestion)
        congestion = sum(congestions)
        return congestion        

In [36]:
def create_sampling_tables(sect_names, sect_sampling_coeff):
    
    sections, origin_sample_table, dest_sample_table = list(), list(), list()

    for idx, s in enumerate(sect_names):
        sections.append(Section(s))
        origin_sample_table.extend([sections[-1]]*sect_sampling_coeff[idx][0])
        dest_sample_table.extend([sections[-1]]*sect_sampling_coeff[idx][1])
    
    return sections, origin_sample_table, dest_sample_table

In [37]:
def generate_population(nr_vehicles, origin_sample_table, dest_sample_table):
    vehicles = list()
    for i in range(nr_vehicles):
        vehicle_type = "car" if np.random.rand() > 0.15 else "bus"
        origin, destination = np.random.choice(origin_sample_table), np.random.choice(dest_sample_table)
        while destination == origin: destination = np.random.choice(dest_sample_table)
        vehicles.append(Vehicle(vehicle_type, origin, destination))
    return vehicles

In [38]:
sect_names = ["AveHub","BusBay","CarCove","DashDale","EcoEnd","FastField","GearBay","HighHill","JamHub","KerbKey","LaneEnd"]
# How often these sections should be origins and destinations?
sect_sampling_coeff = [(3, 7), (2, 8), (9, 1), (7, 4), (10, 1), (4, 2), (8, 1), (3, 3), (1, 10), (6, 3), (10, 2)]

# Road origins and destinations, free flow durations and capacities
roads = [["LaneEnd", "FastField", 2/60, 300], ["FastField", "HighHill", 8/60, 800], ["HighHill", "GearBay", 6/60, 500],\
         ["FastField", "JamHub", 13/60, 400], ["HighHill", "BusBay", 10/60, 500], ["GearBay", "BusBay", 15/60, 600], \
         ["GearBay", "CarCove", 25/60, 1000], ["AveHub", "JamHub", 3/60, 250], ["JamHub", "BusBay", 4/60, 250], \
         ["AveHub", "EcoEnd", 15/60, 600], ["AveHub", "KerbKey", 7/60, 300], ["JamHub", "DashDale", 9/60, 450], \
         ["EcoEnd", "KerbKey", 4/60, 250], ["DashDale", "CarCove", 3/60, 300]]

In [39]:
sections, origin_sample_table, dest_sample_table = create_sampling_tables(sect_names, sect_sampling_coeff)
vehicles = generate_population(population, origin_sample_table, dest_sample_table)

city = City(sections, 0.05)
for road in roads:
    city.build_road(road[0], road[1], road[2], road[3])

In [40]:
print("--- Sections ---")
for s in city.sections:
    print(s.name, "neighbors to:", [x.name for x in s.neighbors])
    
print("\n--- Some of the vehicles ---")
for v in vehicles[1000:1010]:
    print("%s is now at %s. It is going from %s to %s"%(v.vehicle_type, v.location.name, v.origin.name, v.destination.name))

--- Sections ---
AveHub neighbors to: ['JamHub', 'EcoEnd', 'KerbKey']
BusBay neighbors to: ['HighHill', 'GearBay', 'JamHub']
CarCove neighbors to: ['GearBay', 'DashDale']
DashDale neighbors to: ['JamHub', 'CarCove']
EcoEnd neighbors to: ['AveHub', 'KerbKey']
FastField neighbors to: ['LaneEnd', 'HighHill', 'JamHub']
GearBay neighbors to: ['HighHill', 'BusBay', 'CarCove']
HighHill neighbors to: ['FastField', 'GearBay', 'BusBay']
JamHub neighbors to: ['FastField', 'AveHub', 'BusBay', 'DashDale']
KerbKey neighbors to: ['AveHub', 'EcoEnd']
LaneEnd neighbors to: ['FastField']

--- Some of the vehicles ---
car is now at FastField. It is going from FastField to KerbKey
car is now at EcoEnd. It is going from EcoEnd to DashDale
car is now at LaneEnd. It is going from LaneEnd to AveHub
car is now at EcoEnd. It is going from EcoEnd to AveHub
car is now at CarCove. It is going from CarCove to HighHill
car is now at KerbKey. It is going from KerbKey to DashDale
car is now at GearBay. It is going fro

In [41]:
origin_dest_counter = {x:[0, 0] for x in sect_names}
for v in vehicles:
    origin_dest_counter[v.origin.name][0] += 1
    origin_dest_counter[v.destination.name][1] += 1
    
max_dest, max_dest_nr, max_origin, max_origin_nr = None, 0, None, 0
for sect, values in origin_dest_counter.items():
    org, dest = values
    if org > max_origin_nr:
        max_origin_nr = org
        max_origin = sect
    if dest > max_dest_nr:
        max_dest_nr = dest
        max_dest = sect

print(max_origin, "is the most popular origin with", max_origin_nr, "vehicles.\n",\
      max_dest, "is the most popular destination with", max_dest_nr, "vehicles.")

EcoEnd is the most popular origin with 825 vehicles.
 JamHub is the most popular destination with 1245 vehicles.


In [42]:
print("\n--- Some of the vehicles ---")
for v in vehicles[1000:1010]:
    print("%s is now at %s. It is going from %s to %s"%(v.vehicle_type, v.location.name, v.origin.name, v.destination.name))

print("\n",city.get_congestion())

for v in vehicles:
    city.move_vehicle(v, np.random.choice(v.location.neighbors))
    
print(city.get_congestion())

for v in vehicles:
    city.move_vehicle(v, np.random.choice(v.location.neighbors))
    
print(city.get_congestion())

for v in vehicles:
    city.move_vehicle(v, np.random.choice(v.location.neighbors))
    
print(city.get_congestion())

    
print("\n--- Some of the vehicles ---")
for v in vehicles[1000:1010]:
    print("%s is now at %s. It is going from %s to %s"%(v.vehicle_type, v.location.name, v.origin.name, v.destination.name))


--- Some of the vehicles ---
car is now at FastField. It is going from FastField to KerbKey
car is now at EcoEnd. It is going from EcoEnd to DashDale
car is now at LaneEnd. It is going from LaneEnd to AveHub
car is now at EcoEnd. It is going from EcoEnd to AveHub
car is now at CarCove. It is going from CarCove to HighHill
car is now at KerbKey. It is going from KerbKey to DashDale
car is now at GearBay. It is going from GearBay to JamHub
car is now at DashDale. It is going from DashDale to CarCove
bus is now at EcoEnd. It is going from EcoEnd to HighHill
car is now at LaneEnd. It is going from LaneEnd to JamHub

 4.333333333333333
8.769215326388888
6.5076473759259255
6.531539099074073

--- Some of the vehicles ---
car is now at JamHub. It is going from FastField to KerbKey
car is now at EcoEnd. It is going from EcoEnd to DashDale
car is now at FastField. It is going from LaneEnd to LaneEnd
car is now at AveHub. It is going from EcoEnd to AveHub
car is now at CarCove. It is going from 